In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

data_path = '../data/industrial_load_hourly.parquet'
prophet_model_path = '../models/prophet_model.pkl'
lstm_model_path = '../models/lstm_residual_model.h5'
hybrid_model_path = '../models/hybrid_prophet_lstm_model.pkl'

m_prophet = joblib.load(prophet_model_path)
m_lstm = load_model(lstm_model_path)

df = pd.read_parquet(data_path)
df = df.reset_index().rename(columns={'timestamp': 'ds', 'load': 'y'})

split_date = pd.to_datetime('2014-01-01 00:00:00')

train_data = df[df['ds'] < split_date].copy()

In [ ]:
prophet_train_forecast = m_prophet.predict(train_data[['ds']])

train_data['yhat_prophet'] = prophet_train_forecast['yhat']
train_data['residual'] = train_data['y'] - train_data['yhat_prophet']

residual_scaler = MinMaxScaler(feature_range=(0, 1))
train_data['scaled_residual'] = residual_scaler.fit_transform(train_data[['residual']])

TIME_STEPS = m_lstm.input_shape[1] 

def create_residual_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

residual_scaled_values = train_data['scaled_residual'].values
X_res, y_res = create_residual_sequences(residual_scaled_values, TIME_STEPS)
X_res = X_res.reshape(X_res.shape[0], X_res.shape[1], 1)

In [ ]:
test_data_ds = df[df['ds'] >= split_date].copy()
prophet_test_forecast = m_prophet.predict(test_data_ds[['ds']])
test_data_ds['yhat_prophet'] = prophet_test_forecast['yhat']

last_train_residuals = residual_scaled_values[-TIME_STEPS:]

test_scaled_residuals = []
current_batch = last_train_residuals.reshape(1, TIME_STEPS, 1)

for i in range(len(test_data_ds)):

    prediction_output = m_lstm.predict(current_batch, verbose=0)

    predicted_residual_scaled = prediction_output[0, 0]
    
    test_scaled_residuals.append(predicted_residual_scaled)

    current_batch = np.append(
        current_batch[:, 1:, :],
        [[[predicted_residual_scaled]]],
        axis=1
    )

print("Residual prediction loop complete.")

predicted_residuals = residual_scaler.inverse_transform(np.array(test_scaled_residuals).reshape(-1, 1))

test_data_ds['residual_lstm'] = predicted_residuals.flatten()
test_data_ds['yhat_hybrid'] = test_data_ds['yhat_prophet'] + test_data_ds['residual_lstm']

hybrid_components = {
    'prophet_model': m_prophet,
    'residual_lstm': m_lstm,
    'residual_scaler': residual_scaler
}

model_dir = '../models/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

joblib.dump(hybrid_components, hybrid_model_path)

print(f"\nHybrid Model components saved to: {hybrid_model_path}")
print("\nFirst 5 Hybrid Predictions:")
print(test_data_ds[['ds', 'y', 'yhat_hybrid']].head())